# Module 2. GluonTS Training on Amazon SageMaker
---

본 모듈에서는 Amazon SageMaker API를 호출하여 모델 훈련을 수행합니다. 노트북 실행에는 약 10분 가량 소요되며, 핸즈온 실습 시에는 25분을 권장드립니다.

Amazon SageMaker는 완전관리형 머신 러닝 서비스로 인프라 관리에 대해 걱정할 필요가 없으며, 딥러닝 프레임워크의 훈련/배포 컨테이너 이미지를 가져 와서
여러분의 스크립트 코드를 쉽게 통합할 수 있습니다.

<br>

## 1. Training script
---

아래 코드 셀은 `src` 디렉토리에 SageMaker 훈련 스크립트인 `train.py`를 저장합니다.
아래 스크립트가 이전 모듈의 코드와 대부분 일치하다는 점을 알 수 있습니다. 다시 말해, SageMaker 훈련 스크립트 파일은 기존 온프레미스에서 사용했던 Python 스크립트 파일과 크게 다르지 않으며, SageMaker 훈련 컨테이너에서 수행하기 위한 추가적인 환경 변수들만 설정하시면 됩니다.

환경 변수 설정의 code snippet은 아래과 같습니다.

```python
# SageMaker Container environment
parser.add_argument('--model_dir', type=str, default=os.environ['SM_MODEL_DIR'])
parser.add_argument('--data_dir', type=str, default=os.environ['SM_CHANNEL_TRAINING'])
parser.add_argument('--num_gpus', type=int, default=os.environ['SM_NUM_GPUS'])
parser.add_argument('--output_dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))
```

In [6]:
%%writefile ./src/train.py

import os
import pandas as pd
import gluonts 
import numpy as np
import argparse
import json
import pathlib
from mxnet import gpu, cpu
from mxnet.context import num_gpus
import matplotlib.pyplot as plt

from gluonts.dataset.util import to_pandas
from gluonts.mx.distribution import DistributionOutput, StudentTOutput, NegativeBinomialOutput, GaussianOutput
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions, backtest_metrics
from gluonts.model.predictor import Predictor
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset

def train(args):
    
    # Parse arguments
    epochs = args.epochs
    pred_length = args.pred_length
    num_layers = args.num_layers
    num_cells = args.num_cells
    dropout_rate = args.dropout_rate
    batch_size = args.batch_size
    lr = args.lr
    model_dir = args.model_dir
    data_dir = args.data_dir
    num_gpus = args.num_gpus
    output_dir = args.output_dir
    device = "gpu" if num_gpus > 0 else "cpu"
    FREQ = 'D'    
    
    # Get training data
    target_df = pd.read_csv(os.path.join(data_dir, 'target_train.csv'))
    target_df.set_index(target_df.columns[0], inplace=True)    
    target = target_df.values
    num_steps, num_series = target_df.shape
    start_dt = target_df.index[0]
    
    custom_ds_metadata = {'num_series': num_series,
                          'num_steps': num_steps,
                          'prediction_length': pred_length,
                          'freq': FREQ,
                          'start': [start_dt for _ in range(num_series)] 
                         }    

    # Prepare GlounTS Dataset
    train_lst = []
    for i in range(0, num_series):
        target_vec = target[:-pred_length, i]
        dic = {FieldName.TARGET: target_vec, 
               FieldName.START: start_dt} 

        train_lst.append(dic)

    test_lst = []
    for i in range(0, num_series):
        target_vec = target[:, i]
        dic = {FieldName.TARGET: target_vec, 
               FieldName.START: start_dt} 
        test_lst.append(dic)
        
    train_ds = ListDataset(train_lst, freq=FREQ)
    test_ds = ListDataset(test_lst, freq=FREQ)
    train_entry = next(iter(train_ds))
    train_entry.keys()
    
    # Define Estimator    
    trainer = Trainer(
        ctx=device,
        epochs=epochs,
        learning_rate=lr,
        batch_size=batch_size
    )        
    
    deepar_estimator = DeepAREstimator(freq=FREQ, 
                                   prediction_length=pred_length,
                                   num_cells=num_cells,
                                   dropout_rate=dropout_rate,
                                   num_layers=num_layers,
                                   distr_output=StudentTOutput(),
                                   trainer=trainer)

    # Train the model
    deepar_predictor = deepar_estimator.train(train_ds)
    
    # Evaluate trained model on test data
    forecast_it, ts_it = make_evaluation_predictions(test_ds, deepar_predictor, num_samples=100)
    forecasts = list(forecast_it)
    tss = list(ts_it)
    evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
    agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds))

    metrics = ['RMSE', 'MAPE', 'wQuantileLoss[0.1]', 'wQuantileLoss[0.5]', 'wQuantileLoss[0.9]', 'mean_wQuantileLoss']
    metrics_dic = dict((key,value) for key, value in agg_metrics.items() if key in metrics)
    print(json.dumps(metrics_dic, indent=2))

    # Save the model
    deepar_predictor.serialize(pathlib.Path(model_dir))
    return deepar_predictor


def parse_args():
    parser = argparse.ArgumentParser()
    
    # Hyperparameter Setting
    parser.add_argument('--epochs', type=int, default=10)
    parser.add_argument('--pred_length', type=int, default=28)    
    parser.add_argument('--num_layers', type=int, default=2)
    parser.add_argument('--num_cells', type=int, default=30)
    parser.add_argument('--dropout_rate', type=float, default=0.1)
    parser.add_argument('--batch_size', type=float, default=32)
    parser.add_argument('--lr', type=float, default=0.001) 
    
    # SageMaker Container Environment
    parser.add_argument('--model_dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--data_dir', type=str, default=os.environ['SM_CHANNEL_TRAINING'])
    parser.add_argument('--num_gpus', type=int, default=os.environ['SM_NUM_GPUS'])
    parser.add_argument('--output_dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))
    
    args = parser.parse_args()
    return args    

if __name__ == '__main__':
    args = parse_args()
    train(args)    

Overwriting ./src/train.py


<br>

## 2. Training
---

스크립트가 준비되었다면 SageMaker 훈련을 수행하는 법은 매우 간단합니다. SageMaker Python SDK 활용 시, Estimator 인스턴스를 생성하고 해당 인스턴스의 `fit()` 메서드를 호출하는 것이 전부입니다. 좀 더 자세히 기술해 보면 아래와 같습니다.

#### 1) Estimator 인스턴스 생성
훈련 컨테이너에 필요한 설정들을 지정합니다. 본 핸즈온에서는 훈련 스크립트 파일이 포함된 경로인 소스 경로와(source_dir)와 훈련 스크립트 Python 파일만 엔트리포인트(entry_point)로 지정해 주면 됩니다.

#### 2) `fit()` 메서드 호출
`estimator.fit(YOUR_TRAINING_DATA_URI)` 메서드를 호출하면, 훈련에 필요한 인스턴스를 시작하고 컨테이너 환경을 시작합니다. 필수 인자값은 훈련 데이터가 존해자는 S3 경로(`s3://`)이며, 로컬 모드로 훈련 시에는 S3 경로와 로컬 경로(`file://`)를 모두 지정할 수 있습니다.

인자값 중 wait은 디폴트 값으로 `wait=True`이며, 모든 훈련 작업이 완료될 때까지 코드 셀이 freezing됩니다. 만약 다른 코드 셀을 실행하거나, 다른 훈련 job을 시작하고 싶다면 `wait=False`로 설정하여 Asynchronous 모드로 변경하면 됩니다.

**SageMaker 훈련이 끝나면 컨테이너 환경과 훈련 인스턴스는 자동으로 삭제됩니다.** 이 때, SageMaker는 자동으로 `SM_MODEL_DIR` 경로에 저장된 최종 모델 아티팩트를 `model.tar.gz`로 압축하여 훈련 컨테이너 환경에서 S3 bucket으로 저장합니다. 당연히, S3 bucket에 저장된 모델 아티팩트를 다운로드받아 로컬 상에서 곧바로 테스트할 수 있습니다.

In [7]:
import os
import boto3
import sagemaker
from sagemaker.mxnet import MXNet

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
role = sagemaker.get_execution_role()
bucket = sagemaker.Session().default_bucket()

### Upload data to Amazon S3

Amazon SageMaker로 모델 훈련을 실행하기 위해, 데이터를 S3에 업로드합니다. 참고로, 로컬 모드에서 테스트 시에는 S3에 업로드할 필요 없이 로컬 상에서도 훈련이 가능합니다.

In [8]:
prefix = 'timeseries-hol/store-item-demand/train'
s3_bucket = boto3.Session().resource('s3').Bucket(bucket)

s3_bucket.Object(os.path.join(prefix, 'target_train.csv')).upload_file('data/target_train.csv')

In [9]:
estimator = MXNet(entry_point='train.py',
                    source_dir='src',
                    role=role,
                    instance_type='local',
                    instance_count=1,
                    framework_version='1.6.0',
                    py_version='py3',
                    hyperparameters = {'epochs': 2, 
                                       'lr': 0.001,
                                       'pred_length': 28
                                      }                       
                   )

로컬 파일 시스템에서 직접 훈련을 수행해 보겠습니다.

In [10]:
local_file = f'file://{os.getcwd()}/data/target_train.csv'
estimator.fit(local_file)

Creating wa2nyeeh8n-algo-1-k6sdd ... 
Creating wa2nyeeh8n-algo-1-k6sdd ... done
Attaching to wa2nyeeh8n-algo-1-k6sdd
wa2nyeeh8n-algo-1-k6sdd | 2022-07-17 08:13:33,419 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
wa2nyeeh8n-algo-1-k6sdd | 2022-07-17 08:13:33,421 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
wa2nyeeh8n-algo-1-k6sdd | 2022-07-17 08:13:33,435 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1-k6sdd"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{"epochs":2,"lr":0.001,"pred_length":28}', 'SM_USER_ENTRY_POINT': 'train.py', 'SM_FRAMEWORK_PARAMS': '{}', 'SM_RESOURCE_CONFIG': '{"current_host":"algo-1-k6sdd","hosts":["algo-1-k6sdd"]}', 'SM_INPUT_DATA_CONFIG': '{"training":{"TrainingInputMode":"File"}}', 'SM_OUTPUT_DATA_DIR': '/opt/ml/output/data', 'SM_CHANNELS': '["training"]', 'SM_CURRENT_HOST': 'algo-1-k6sdd', 'SM_MODULE_NAME': 'train', 'SM_LOG_LEV

물론 S3 경로에서도 로컬 모드 훈련이 가능합니다. 

In [11]:
s3_input = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}'.format(bucket, prefix))
estimator.fit(s3_input)

Creating 4pd955oad8-algo-1-cxc3y ... 
Creating 4pd955oad8-algo-1-cxc3y ... done
Attaching to 4pd955oad8-algo-1-cxc3y
4pd955oad8-algo-1-cxc3y | 2022-07-17 08:14:10,061 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
4pd955oad8-algo-1-cxc3y | 2022-07-17 08:14:10,064 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
4pd955oad8-algo-1-cxc3y | 2022-07-17 08:14:10,078 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1-cxc3y"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{"epochs":2,"lr":0.001,"pred_length":28}', 'SM_USER_ENTRY_POINT': 'train.py', 'SM_FRAMEWORK_PARAMS': '{}', 'SM_RESOURCE_CONFIG': '{"current_host":"algo-1-cxc3y","hosts":["algo-1-cxc3y"]}', 'SM_INPUT_DATA_CONFIG': '{"training":{"TrainingInputMode":"File"}}', 'SM_OUTPUT_DATA_DIR': '/opt/ml/output/data', 'SM_CHANNELS': '["training"]', 'SM_CURRENT_HOST': 'algo-1-cxc3y', 'SM_MODULE_NAME': 'train', 'SM_LOG_LEV

### SageMaker Hosted Training

훈련 코드가 로컬에서 잘 작동하므로, 이제 SageMaker에서 관리하는 훈련 인스턴스를 사용하여 훈련을 수행하겠습니다. 로컬 모드 훈련과 달리 호스팅 훈련은
노트북 인스턴스 대신에 SageMaker에서 관리하는 별도의 클러스터에서 수행합니다. 본 핸즈온의 데이터셋 사이즈가 작기 때문에 체감이 되지 않겠지만, 대규모 데이터 및 복잡한 모델에 대한 분산 훈련은 SageMaker 호스팅 훈련 방법을 사용하는 것을 권장합니다.

In [12]:
estimator = MXNet(entry_point='train.py',
                  source_dir='src',
                  role=role,
                  instance_type='ml.c5.xlarge',
                  instance_count=1,
                  framework_version='1.6.0',
                  py_version='py3',
                  hyperparameters = {'epochs': 20, 
                                     'lr': 0.001,
                                     'pred_length': 28                                     
                                    }                       
                 )

In [13]:
estimator.fit(s3_input)

2022-07-17 08:14:50 Starting - Starting the training job...
2022-07-17 08:15:07 Starting - Preparing the instances for trainingProfilerReport-1658045690: InProgress
............
2022-07-17 08:17:11 Downloading - Downloading input data...
2022-07-17 08:17:51 Training - Training image download completed. Training in progress..2022-07-17 08:17:51,839 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
2022-07-17 08:17:51,842 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-07-17 08:17:51,859 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{"epochs":20,"lr":0.001,"pred_length":28}', 'SM_USER_ENTRY_POINT': 'train.py', 'SM_FRAMEWORK_PARAMS': '{}', 'SM_RESOURCE_CONFIG': '{"current_group_name":"homogeneousCluster","current_host":"algo-1","current_instance_type":"ml.c5.xlarge","hosts":["algo-1"],"instance_groups":[{"hosts

<br>

## 3. Getting Model Artifacts
---

훈련이 완료된 모델 아티팩트를 로컬(노트북 인스턴스 or 온프레미스)로 복사합니다. 훈련 완료 시 `SM_MODEL_DIR`에 있는 파일들이
`model.tar.gz`로 자동으로 압축되며, 압축을 해제하여 로컬 상에서도 추론을 수행할 수 있습니다.

In [14]:
local_model_dir = './model'
!rm -rf $local_model_dir

In [15]:
import json, os

s3_model_dir = estimator.model_data.replace('model.tar.gz', '')
print(s3_model_dir)
!aws s3 ls {s3_model_dir}

if not os.path.exists(local_model_dir):
    os.makedirs(local_model_dir)

!aws s3 cp {s3_model_dir}model.tar.gz {local_model_dir}/model.tar.gz
!tar -xzf {local_model_dir}/model.tar.gz -C {local_model_dir}

s3://sagemaker-us-east-1-057716757052/mxnet-training-2022-07-17-08-14-50-371/output/
2022-07-17 08:19:49      60060 model.tar.gz
download: s3://sagemaker-us-east-1-057716757052/mxnet-training-2022-07-17-08-14-50-371/output/model.tar.gz to model/model.tar.gz


다음 모듈에서 활용할 변수들을 저장합니다.

In [16]:
%store s3_model_dir
%store prefix

Stored 's3_model_dir' (str)
Stored 'prefix' (str)
